# Week 5 Capstone Project

## Introduction/Business Problem

I will look into which neighborhood in Brooklyn, NYC would be the best option to open a mexican food restaurant based on the restaurants already in the neighborhood and cuisine served by the restaurants. To open a profitable restaurant we need to find an area that isn't densely populated with restaurant and contain too many that offer mexican food. 

## Data Section

Data will be pulled in using the foursquare API to pull in a list of venues in each neighborhood. The data will be complied and clustered using KNN to find which have the most restaurants, the cuisine of the restaurant, and the general make up of the neighborhood. Some neighborhoods may satisfy the criteria of few restaurants and cuisine but may not be an ideal location for a restaurant such as a empty plot of land. 

Once the data is collected and analzyed a recommendation will be given with the top 3 neighborhood to open up a restaurant.

# Methodology Section

The data being used is the NYC neighborhood dataset provided by the course.

The libraries used for this analysis were numpy, pandas, json, geopy, matplotlib, sklearn, and folium. Along with the foursqaure API.

The neighborhood data was imported and cleaned up using the libraries. 

The neighborhoods in Brooklyn were separated from the rest of the dataset that contained all neighborhoods in NYC.

The foursquare API was then used to import the venues of each neighborhood in Brooklyn.

After which KNN was used to cluster the neighborhoods into 5 separate clusters.




In [1]:
# Import libaries needed

import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

In [2]:
# Download data on NYC neighborhoods 

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
# Turn data into a pandas datafram
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="nyc")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [10]:

map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

Separate Brooklyn from the rest of the data.

In [11]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [12]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="nyc")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


Create a map of the neighborhoods in Brooklyn.

In [13]:
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

Set up Foursquare API and pull in venues from the neighborhoods.

In [41]:
CLIENT_ID = 'XXXXX' 
CLIENT_SECRET = 'XXXXX' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXXXX
CLIENT_SECRET:XXXXX


In [15]:
brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

In [16]:
neighborhood_latitude = brooklyn_data.loc[0, 'Latitude'] 
neighborhood_longitude = brooklyn_data.loc[0, 'Longitude'] 

neighborhood_name = brooklyn_data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [17]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=N5HC125TQFMVTZ5DP5QE3RAJTVED1E2MOG033YLGBPHLM4UD&client_secret=KDPMBMEM3KEDOGV1GGEPGSUGJJDBUXZG03E14H5OOIZM0L4E&v=20180605&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

In [18]:
results = requests.get(url).json()


In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Leo's Casa Calamari,Pizza Place,40.624200,-74.030931
3,Cocoa Grinder,Juice Bar,40.623967,-74.030863
4,Ho' Brah Taco Joint,Taco Place,40.622960,-74.031371


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

86 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )



Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [24]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2729, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [25]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,47,47,47,47,47,47
Bay Ridge,86,86,86,86,86,86
Bedford Stuyvesant,30,30,30,30,30,30
Bensonhurst,35,35,35,35,35,35
Bergen Beach,5,5,5,5,5,5
Boerum Hill,87,87,87,87,87,87
Borough Park,22,22,22,22,22,22
Brighton Beach,44,44,44,44,44,44
Broadway Junction,17,17,17,17,17,17


In [26]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 285 uniques categories.


In [27]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaur

In [28]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaur

Find top 10 venues in each neighborhood

In [29]:
num_top_venues = 10

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0           Pizza Place  0.06
1              Pharmacy  0.06
2           Gas Station  0.04
3  Cantonese Restaurant  0.04
4    Chinese Restaurant  0.04
5  Fast Food Restaurant  0.04
6    Italian Restaurant  0.04
7            Donut Shop  0.04
8       Bubble Tea Shop  0.04
9        Cosmetics Shop  0.02


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.08
1                  Spa  0.07
2          Pizza Place  0.05
3           Bagel Shop  0.05
4        Grocery Store  0.03
5                  Bar  0.03
6     Greek Restaurant  0.03
7  American Restaurant  0.03
8             Pharmacy  0.02
9           Hookah Bar  0.02


----Bedford Stuyvesant----
                    venue  freq
0           Deli / Bodega  0.10
1             Coffee Shop  0.10
2             Pizza Place  0.07
3                     Bar  0.07
4                    Café  0.07
5               Juice Bar  0.03
6             Bus Station  0.03
7  Thrift / Vintage Store  

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Pizza Place,Bubble Tea Shop,Italian Restaurant,Gas Station,Donut Shop,Fast Food Restaurant,Cantonese Restaurant,Chinese Restaurant,Liquor Store
1,Bay Ridge,Italian Restaurant,Spa,Bagel Shop,Pizza Place,Grocery Store,Greek Restaurant,American Restaurant,Bar,Playground,Sandwich Place
2,Bedford Stuyvesant,Deli / Bodega,Coffee Shop,Bar,Café,Pizza Place,Tiki Bar,New American Restaurant,Fried Chicken Joint,Boutique,Gift Shop
3,Bensonhurst,Chinese Restaurant,Italian Restaurant,Pizza Place,Ice Cream Shop,Sushi Restaurant,Donut Shop,Bakery,Supermarket,Sporting Goods Shop,Liquor Store
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Playground,Baseball Field,Women's Store,Field,Factory,Falafel Restaurant,Farm,Farmers Market


Cluster the neighborhoods using KNN

In [32]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 4, 1, 4, 1], dtype=int32)

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,1,Italian Restaurant,Spa,Bagel Shop,Pizza Place,Grocery Store,Greek Restaurant,American Restaurant,Bar,Playground,Sandwich Place
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Chinese Restaurant,Italian Restaurant,Pizza Place,Ice Cream Shop,Sushi Restaurant,Donut Shop,Bakery,Supermarket,Sporting Goods Shop,Liquor Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Pizza Place,Mobile Phone Shop,Mexican Restaurant,Latin American Restaurant,Bank,Bakery,Gym,Pharmacy,Fried Chicken Joint,Creperie
3,Brooklyn,Greenpoint,40.730201,-73.954241,1,Bar,Pizza Place,Cocktail Bar,Coffee Shop,French Restaurant,Mexican Restaurant,Deli / Bodega,Sushi Restaurant,Yoga Studio,Tea Room
4,Brooklyn,Gravesend,40.595260,-73.973471,1,Lounge,Pizza Place,Chinese Restaurant,Italian Restaurant,Bakery,Furniture / Home Store,Gym,Breakfast Spot,Metro Station,Farmers Market


# Results

Map of clusters color key

Cluster 0: Red

Cluster 1: Purple

Cluster 2: Blue

Cluster 3: Green 

Cluster 4: Orange

In [40]:
# create map
map_bk = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_bk)
       
map_bk

Cluster 0

In [35]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Bergen Beach,Harbor / Marina,Athletics & Sports,Playground,Baseball Field,Women's Store,Field,Factory,Falafel Restaurant,Farm,Farmers Market


Cluster 1

In [36]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Spa,Bagel Shop,Pizza Place,Grocery Store,Greek Restaurant,American Restaurant,Bar,Playground,Sandwich Place
1,Bensonhurst,Chinese Restaurant,Italian Restaurant,Pizza Place,Ice Cream Shop,Sushi Restaurant,Donut Shop,Bakery,Supermarket,Sporting Goods Shop,Liquor Store
2,Sunset Park,Pizza Place,Mobile Phone Shop,Mexican Restaurant,Latin American Restaurant,Bank,Bakery,Gym,Pharmacy,Fried Chicken Joint,Creperie
3,Greenpoint,Bar,Pizza Place,Cocktail Bar,Coffee Shop,French Restaurant,Mexican Restaurant,Deli / Bodega,Sushi Restaurant,Yoga Studio,Tea Room
4,Gravesend,Lounge,Pizza Place,Chinese Restaurant,Italian Restaurant,Bakery,Furniture / Home Store,Gym,Breakfast Spot,Metro Station,Farmers Market
5,Brighton Beach,Beach,Eastern European Restaurant,Restaurant,Russian Restaurant,Sushi Restaurant,Gourmet Shop,Mobile Phone Shop,Bank,Fast Food Restaurant,Dessert Shop
6,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Sandwich Place,Yoga Studio,Miscellaneous Shop,Café,Outlet Store,Russian Restaurant,Grocery Store,Restaurant
7,Manhattan Terrace,Ice Cream Shop,Pizza Place,Grocery Store,Donut Shop,Spa,Mobile Phone Shop,Organic Grocery,Liquor Store,Steakhouse,Bank
8,Flatbush,Caribbean Restaurant,Mexican Restaurant,Coffee Shop,Deli / Bodega,Juice Bar,Lounge,Chinese Restaurant,Grocery Store,Bank,Bagel Shop
9,Crown Heights,Pizza Place,Museum,Café,Coffee Shop,Moving Target,Fried Chicken Joint,Bookstore,Supermarket,Candy Store,Salon / Barbershop


Cluster 2

In [37]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Women's Store,Filipino Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop


Cluster 3

In [38]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Canarsie,Gym,Chinese Restaurant,Caribbean Restaurant,Asian Restaurant,Home Service,Thai Restaurant,Fast Food Restaurant,Factory,Falafel Restaurant,Farm
59,Paerdegat Basin,Home Service,Food,Asian Restaurant,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


Cluster 4

In [39]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,East Flatbush,Moving Target,Liquor Store,Pharmacy,Park,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Food & Drink Shop,Department Store,Supermarket
14,Brownsville,Chinese Restaurant,Plaza,Convenience Store,Moving Target,Park,Fried Chicken Joint,Performing Arts Venue,Restaurant,Trail,Pharmacy
25,Cypress Hills,Fast Food Restaurant,Ice Cream Shop,Fried Chicken Joint,Chinese Restaurant,Donut Shop,Supermarket,Baseball Field,Liquor Store,Caribbean Restaurant,Gas Station
26,East New York,Deli / Bodega,Fast Food Restaurant,Pizza Place,Event Service,Music Venue,Gym,Metro Station,Pharmacy,Bus Stop,Home Service
27,Starrett City,Supermarket,Donut Shop,Pizza Place,Bus Stop,Pharmacy,Bus Station,Caribbean Restaurant,American Restaurant,Fish Market,Flower Shop
29,Flatlands,Pharmacy,Fried Chicken Joint,Caribbean Restaurant,Fast Food Restaurant,Lounge,Chinese Restaurant,Bar,Paper / Office Supplies Store,Park,Arts & Crafts Store
34,Borough Park,Bank,Pharmacy,Fast Food Restaurant,Pizza Place,Café,Restaurant,Men's Store,Farmers Market,Coffee Shop,Grocery Store
47,Prospect Park South,Caribbean Restaurant,Mobile Phone Shop,Pizza Place,Grocery Store,Fast Food Restaurant,Mexican Restaurant,Fried Chicken Joint,Clothing Store,Donut Shop,Pharmacy
55,Wingate,Deli / Bodega,Fast Food Restaurant,Fried Chicken Joint,Park,Juice Bar,Bakery,BBQ Joint,Café,Fish & Chips Shop,Liquor Store
56,Rugby,Caribbean Restaurant,Bank,Grocery Store,Seafood Restaurant,Sandwich Place,Wings Joint,Diner,Pizza Place,Salon / Barbershop,Fried Chicken Joint


# Discussion 

Looking at the data clusters 1 and 4 contain the most neighborhoods which are primarily made up of restaurants.
While Clusters 0,2, and 3 contain fewer restaurants.

Cluster 0 and 2 share similar make up and are mainly made up of venues that consists of sporting or outdoor activities. While it may not contain many restaurant people who are more active may not see Mexican food as the healthiest option for a after workout meal. The areas could also be made up of residential housing so that could make it difficult to find space to put up a restaurant.

Cluster 1 is filled with neighborhoods that consist mainly of restaurants that offer a sit down experience not provided in fast food restaurants. One of the neighborhood in this cluster could be a good choice as long as mexican food isn't in any of the top 10 most common venues and the restaurants plans to not be a fast food joint.

Cluster 3 has a mix of restaurants that tend to favor asian cuisine. The asian cuisine itself is very diverse but a introducing a mexican restaurant can spark interest in a neighborhood where it's known for its mix of asian food.

Cluster 4 is also filled with restaurants but it favors fast food restaurants as they commonly appear in the top 5 of common venues. The other common venues delis, pizza, and fried chicken joint, and chinese restaurant can be considered fast food as they typically aren't known for their sit down experiences. 

Based on this a neighborhood in cluster 1 would be the ideal choice if your also looking to provide a great dinning experience along with good food. Cluster 3 would be the second choice as it mainly focuses on asian cuisine and mexican food can stand out even more. Cluster 3 would be the third choice if your looking to provide quick good food and not focus on the sit down experience. Maybe a food truck can work well here.




# Conclusion 

In conclusion after looking at the data I would recommend Williamsburg, Brooklyn Heights, and Canarsie as viable options for a mexican restaurant.

Williamsburg doesn't have any restaurants in the top 5 but contains a latin american restaurant in the top 10. This could drum up interest in a mexican food restaurant.

Brooklyn Heights contains 3 restaurants in the top 10 with mexican restaurants being top 7. This location doenst have many restaurants and seeing the success of other mexican restaurants in the area can help provide insight.

Canarsie is filled with restaurants but they mainly asian cuisine this can provide a unique opportunity for a mexican restaurant to stick out and gain more attention.

